In [1]:
!pip install transformers
!pip install accelerate -U


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [2]:
!pip install datasets
!pip install rouge_score
!pip install tiktoken

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6abb70e65863fb0034a3aedb8997b634fa393e718e58193dc3d92f40c0f195f9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:0000:0100:010m


In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import statistics
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
import tiktoken

In [4]:
df = pd.read_csv('/kaggle/input/qa-data/aiml-qa-test.csv',encoding='Windows-1252',header=0)  

df= df[['question', 'answer1','answer2']]

print(df.head())

                                            question  \
0    How we can effectively convert 2D images to 1D?   
1  Can we utilize an autoencoder to perform dimen...   
2  What is NLP's current biggest challenge that i...   
3  Which problems cannot be solved by Neural netw...   
4                      Is scaling necessary for SVM?   

                                             answer1  \
0  Converting images to 1D data may not be effect...   
1  Yes, autoencoders can be applied to numerical ...   
2  The main challenges of NLP is finding and coll...   
3  While neural networks have shown great success...   
4  Yes, scaling the input data is generally recom...   

                                             answer2  
0  To effectively convert 2D images to 1D, use te...  
1  Yes, autoencoders can be used for dimensionali...  
2  NLP models struggle with tasks that require re...  
3  Neural networks are powerful, but they may str...  
4  Scaling the input data is advisable when utili..

In [46]:

print(df.isnull().sum())
df.dropna(subset=['question', 'answer1','answer2'], inplace=True)
print(df.isnull().sum())

question    0
answer1     0
answer2     0
dtype: int64
question    0
answer1     0
answer2     0
dtype: int64


In [38]:
pd.set_option('display.max_colwidth', None)

df.head(3)

,question,answer
0,What is the difference between concatenation vs. summation of two tensors?,Concatenation combines two tensors by adding them together along a specified dimension. Summation adds the elements of two tensors together element-wise.
1,What is the difference between concatenation vs. summation of two tensors?,"Concatenation is often used to combine different features of a data set, such as images and text. Summation is often used to calculate the similarity between two tensors, or to compute the loss function for a neural network."
2,Why are derivatives substracted from weights?,"The derivative of the loss function at a point indicates the direction of steepest descent from that point, substrating the derivative from the weights helps in finding the lowest point in the loss function."


In [13]:
import torch
from safetensors.torch import load_file as load_safetensors

In [43]:
model_path = "/kaggle/input/gpt2qafine/tensorflow2/default/1/kaggle/working/saved_model"
state_dict = load_safetensors(f"{model_path}/model.safetensors")
model = GPT2LMHeadModel.from_pretrained(model_path, state_dict=state_dict)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [24]:
df.shape

(120, 3)

In [21]:
data_test = []
for index, row in df.iterrows():
    data_test += [{'question': row.iloc[0], 'answer1': row.iloc[1], 'answer2': row.iloc[2]}]


In [22]:
import json
with open('test.json', 'w') as file:
    json.dump(data_test, file)

In [45]:
# Tokenize the dataset
from datasets import load_dataset
dataset = load_dataset('json', data_files = {'test':'test.json'})
tokenizer.pad_token = tokenizer.eos_token
def preprocess_function(examples):
    inputs = [q + tokenizer.eos_token + a1 + tokenizer.eos_token + a2 for q, a1, a2 in zip(examples['question'], examples['answer1'], examples['answer2'])]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = model_inputs['input_ids'].copy()
    model_inputs['labels'] = labels
    return model_inputs

tokenized_dataset1 = dataset.map(preprocess_function, batched=True)

In [48]:


generated_answers = []
for example in data_test:
    input_ids = tokenizer.encode(example['question'], return_tensors='pt').to(device)
    attention_mask = torch.ones(input_ids.shape, device=device)
    output = model.generate(input_ids, attention_mask = attention_mask, max_length=100)
    generated_answer = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_answers.append(generated_answer)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [51]:

from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []
for i, example in enumerate(data_test):
    target_answer = example['answer1']
    generated_answer = generated_answers[i]
    scores = scorer.score(target_answer, generated_answer)
    rouge_scores.append(scores)


avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1 Score: {avg_rouge1}")
print(f"Average ROUGE-2 Score: {avg_rouge2}")
print(f"Average ROUGE-L Score: {avg_rougeL}")

Average ROUGE-1 Score: 0.34060178560396165
Average ROUGE-2 Score: 0.15400855994599644
Average ROUGE-L Score: 0.2726937340427868


In [57]:
# Human Evaluation

def question_gen(question):
    inputs = tokenizer.encode(f"Question: {question} Answer: ", return_tensors='pt').to(device)
    attention_mask = torch.ones(inputs.shape, device=device)
    outputs = model.generate(inputs, attention_mask = attention_mask, max_length=150, num_return_sequences=1)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

print(question_gen("What is machine learning?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is machine learning? Answer:  Machine learning is a subset of artificial intelligence that focuses on algorithms and models that can learn from data and make predictions or decisions.


In [26]:
len(data_test)

120